In [1]:
# Using Python 3.11
!pip install --upgrade pip -q
!pip install -r requirements.txt -q

In [5]:
import os
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()
openai_key = os.getenv('OPENAI_API_KEY')

client = OpenAI(
    api_key=openai_key,
)

In [6]:
from unstructured.partition.pdf import partition_pdf
import glob
import os
import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

class DocumentPair:
    def __init__(self, input_data, output_data):
        self.input = input_data
        self.output = output_data

def read_pdf(file_path, languages=['deu']):
    """Read and process PDF file content."""
    try:
        return "\n\n".join(str(part) for part in partition_pdf(file_path, languages=languages))
    except FileNotFoundError:
        logging.error(f"PDF file not found: {file_path}")
    except Exception as e:
        logging.error(f"Error processing PDF {file_path}: {e}")
    return None

def read_text(file_path):
    """Read text file content."""
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.read()
    except FileNotFoundError:
        logging.error(f"TXT file not found: {file_path}")
    except Exception as e:
        logging.error(f"Error reading TXT {file_path}: {e}")
    return None

def get_data_pairs(base_path="data/in_context/"):
    """Generate a dictionary of DocumentPair instances from PDF and TXT files, using the folder name as the key."""
    data_pairs = {}
    directories = glob.glob(f"{base_path}*/")
    for directory in directories:
        pdf_file = glob.glob(f"{directory}*.pdf")[0]
        txt_file = glob.glob(f"{directory}*.txt")[0]
        if os.path.exists(pdf_file) and os.path.exists(txt_file):
            folder_name = os.path.basename(os.path.normpath(directory))
            input_data = read_pdf(pdf_file)
            output_data = read_text(txt_file)
            if input_data and output_data:
                data_pairs[folder_name] = DocumentPair(input_data, output_data)
            else:
                logging.warning(f"Data not found for folder: {folder_name}")
        else:
            logging.warning(f"Missing files in folder: {directory}")
    return data_pairs

In [7]:
def format_prompt(input_text, output_text=''):
    """
    Format a single input-output pair into a structured AI training prompt.
    """
    return f"Eingabe:\n{input_text}\n\nAusgabe:\n{output_text if output_text else ''}"

def create_base_prompt(data_pairs):
    """
    Create a base prompt from all document pairs, used as the starting point for each new prompt.
    """
    return '\n'.join(format_prompt(pair.input, pair.output) for _, pair in data_pairs.items())


def create_comprehensive_prompt(base_prompt, folder_path):
    """
    Generate a single comprehensive prompt by appending the content of each new PDF found in the specified directory to the base prompt.
    """
    pdf_paths = glob.glob(os.path.join(folder_path, '*.pdf'))
    non_empty_contents = [content for path in pdf_paths if (content := read_pdf(path))]
    all_pdf_text = format_prompt(' '.join(non_empty_contents))
    return base_prompt + all_pdf_text


# Main execution flow
data_pairs = get_data_pairs(base_path="data/in_context/")
base_prompt = create_base_prompt(data_pairs)
base_path = 'data/tagesordnung'
comprehensive_prompts = [create_comprehensive_prompt(base_prompt, path) for path in glob.glob(os.path.join(base_path, '**'))]

2024-04-26 14:32:02,074 - INFO - pikepdf C++ to Python logger bridge initialized


In [134]:
def create_completion(prompt, gpt_model = "gpt-3.5-turbo-0125"):
  completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model=gpt_model,
    temperature=0.1
  )
  return(completion.choices[0].message.content, completion.usage)

In [135]:

input_prompts = [f"""
Erstelle einen informativen Artikel basierend auf der vorliegenden Tagesordnung. 
Der Artikel sollte den Titel der Veranstaltung hervorheben und die wesentlichen Punkte und Beschlüsse der Sitzung zusammenfassen. 
Ziel ist es, Lesern einen klaren Überblick über die Inhalte und Ergebnisse der Besprechung zu bieten.
Schreibe zuerst die Überschrift im Format (x. Sitzung des Ausschusses y), dann den Artikel, dann Zeit und Ort. Bleibe bitte im Markdown-Format.

Beispiel:                 
{prompt}

Überschrift:
Artikel:
Zeit:
Ort:
---
""" for prompt in comprehensive_prompts]
input_prompts



['\nErstelle einen informativen Artikel basierend auf der vorliegenden Tagesordnung. \nDer Artikel sollte den Titel der Veranstaltung hervorheben und die wesentlichen Punkte und Beschlüsse der Sitzung zusammenfassen. \nZiel ist es, Lesern einen klaren Überblick über die Inhalte und Ergebnisse der Besprechung zu bieten.\nSchreibe zuerst die Überschrift im Format (x. Sitzung des Ausschusses y), dann den Artikel, dann Zeit und Ort. Bleibe bitte im Markdown-Format.\n\nBeispiel:                 \nEingabe:\n20. Wahlperiode\n\nMitteilung\n\nDie 61. Sitzung des Ausschusses für Familie,\n\nSenioren, Frauen und Jugend \x00ndet statt am Mittwoch, dem 13. März 2024, 11:00 Uhr Paul-Löbe-Haus Sitzungssaal: 2.200\n\nDie Sitzung ist öffentlich!\n\nTagesordnung\n\nTagesordnungspunkt 1\n\nFachgespräch zu den Lebensbedingungen von Kindern und Jugendlichen in Unterkünften für ge\x00üchtete Menschen in Deutschland (Studie „Das ist nicht das Leben - Perspektiven von Kindern und Jugendlichen in Unterkünften 

In [130]:
#GPT 3.5
outputs_gpt_three = [create_completion(input_prompt) for input_prompt in input_prompts]

#GPT 4
# outputs_gpt_four = [create_completion(input_prompt, gpt_model="gpt-4-turbo") for input_prompt in input_prompts]


2024-04-23 10:55:57,647 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-04-23 10:56:02,518 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [131]:
from IPython.display import display, Markdown
print("GPT 3.5")
for out in outputs_gpt_three:
    display(Markdown(out[0]))
    print("")
# print("GPT 4")
# for out in outputs_gpt_four:
#     display(Markdown(out[0]))
#     print("")

GPT 3.5


Bildung und Forschung

# 66. Sitzung des Ausschusses für Bildung, Forschung und Technikfolgenabschätzung

Der **Ausschuss für Bildung, Forschung und Technikfolgenabschätzung** hat am **Mittwoch, 13. März 2024,** eine öffentliche Sitzung abgehalten. Neben der Wahl der stellvertretenden Ausschussvorsitzenden stand ein Gespräch mit der Expertenkommission Forschung und Innovation (EFI) zur Vorstellung des EFI-Gutachtens 2024 unter Vorsitz von Herrn Prof. Dr. Uwe Cantner auf der Tagesordnung. Des Weiteren wurden Anträge zur Internationalisierung von Wissenschaft und Hochschulbildung sowie zur Stärkung der deutschen Vermittlerorganisationen in der internationalen Zusammenarbeit diskutiert. (13.03.2024)

**Zeit:**  
Mittwoch, 13. März 2024,  
9:30 Uhr  

**Ort:** Berlin, Marie-Elisabeth-Lüders-Haus, Anhörungssaal (3.101)

Nachhaltige Entwicklung

# 62. Sitzung des Parlamentarischen Beirates für nachhaltige Entwicklung

Der **Parlamentarische Beirat für nachhaltige Entwicklung** traf sich am **Mittwoch, 24. April 2024,** zu einer öffentlichen Sitzung im Paul-Löbe-Haus. Auf der Tagesordnung stand die Nachbereitung der Sitzung des Staatssekretärsausschusses für nachhaltige Entwicklung zum Thema „Forschung, Innovation und Digitalisierung“ durch Dr. Jens Brandenburg, MdB, Parlamentarischer Staatssekretär bei der Bundesministerin für Bildung und Forschung. Des Weiteren wurde über die Beschlussfassung der Votenliste zur Nachhaltigkeitsprüfungsbewertung diskutiert. (24.04.2024)

**Zeit:**  
Mittwoch, 24. April 2024,  
17 Uhr  

**Ort:** Berlin, Paul-Löbe-Haus, Sitzungssaal E.700

In [132]:
def calculate_cost(prompt_tokens, completion_tokens, rates):
    """Calculate the total cost of using GPT-3.5 or GPT-4 Turbo based on prompt and completion tokens."""
    cost_input = prompt_tokens * rates['input']
    cost_output = completion_tokens * rates['output']
    return cost_input + cost_output

def process_outputs(outputs, rates, model_version):
    """Process each output, calculate and print the cost."""
    final_cost = 0
    for i, out in enumerate(outputs):
        prompt_tokens = out[1].prompt_tokens
        completion_tokens = out[1].completion_tokens
        total_cost = calculate_cost(prompt_tokens, completion_tokens, rates)
        final_cost += total_cost
        print(f"The total cost of using {model_version} for query {i} is: ${total_cost:.6f}")
    print(f"The final cost for {model_version} is: €{final_cost:.6f}")

# Rates for GPT-3.5 and GPT-4
rates_gpt_three = {'input': 0.50 / 1_000_000, 'output': 1.50 / 1_000_000}
rates_gpt_four = {'input': 10 / 1_000_000, 'output': 30 / 1_000_000}

# Assuming outputs_gpt_three and outputs_gpt_four are lists of outputs
process_outputs(outputs_gpt_three, rates_gpt_three, "GPT-3.5 Turbo")
# process_outputs(outputs_gpt_four, rates_gpt_four, "GPT-4 Turbo")


The total cost of using GPT-3.5 Turbo for query 0 is: $0.004879
The total cost of using GPT-3.5 Turbo for query 1 is: $0.002063
The final cost for GPT-3.5 Turbo is: €0.006942
